# CASE STUDY - ANALYSING A BOND

Case study for Hyderabad Conference December 2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.finutils import *
from financepy.products.bonds.FinBond import *

####################################################################
# FINANCEPY BETA Version 0.187 - This build:  02 Dec 2020 at 20:12 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



# Define the Bond

In [3]:
issueDate = FinDate(15, 5, 2020)

In [4]:
maturityDate = FinDate(15, 5, 2030)

In [5]:
coupon = 0.050 # This means 5%

In [6]:
freqType = FinFrequencyTypes.ANNUAL

In [7]:
accrualType = FinDayCountTypes.ACT_ACT_ICMA

In [8]:
faceAmount = ONE_MILLION

In [9]:
bond = FinBond(issueDate, maturityDate, coupon, freqType, accrualType, faceAmount)

You can get information about the bond using the print method.

In [10]:
print(bond)

OBJECT TYPE: FinBond
ISSUE DATE: 15-MAY-2020
MATURITY DATE: 15-MAY-2030
COUPON: 0.05
FREQUENCY: FinFrequencyTypes.ANNUAL
ACCRUAL TYPE: FinDayCountTypes.ACT_ACT_ICMA
FACE AMOUNT: 1000000


In [11]:
print(issueDate)

15-MAY-2020


In [12]:
setDateFormatType(FinDateFormatTypes.UK_LONGEST)

In [13]:
print(issueDate)

FRI 15 MAY 2020


## Bond Cash Flows

We first need to set the settlement date of the bond. 

In [ ]:
settlementDate = FinDate(6, 12, 2020)

In [ ]:
bond.printFlows(settlementDate)

The convention is to use these dates for yield calculations even if some fall on weekends.

## Bond Yield Measures

In [ ]:
cleanPrice = 102.20

Yield to maturity using different conventions

In [ ]:
for ytmCalcType in FinYTMCalcType:
    y = bond.yieldToMaturity(settlementDate, cleanPrice, ytmCalcType)
    print("%30s %12.7f" % (ytmCalcType, y*100))

## Vectorizations

Some bond function such as yield to maturity support vectorisation

In [ ]:
cleanPrices = np.linspace(70,150,20)

In [ ]:
ylds = bond.yieldToMaturity(settlementDate, cleanPrices, FinYTMCalcType.US_TREASURY)

In [ ]:
plt.plot(ylds*100.0, cleanPrices)
plt.xlabel("Yields")
plt.ylabel("Clean Prices")

In [ ]:
cleanPrices = bond.cleanPriceFromYTM(settlementDate, ylds, FinYTMCalcType.US_TREASURY)

And so we recover the initial input clean prices.

## Accrued Interest

For consistency let's fix the yield calculation convention to be US Treasury

In [ ]:
yieldConvention = FinYTMCalcType.US_TREASURY

In [ ]:
ytm = bond.yieldToMaturity(settlementDate, cleanPrice, yieldConvention)

Full price is the clean price plus accrued interest

In [ ]:
print("Full Price = %12.7f" % bond.fullPriceFromYTM(settlementDate, ytm, yieldConvention))

In [ ]:
print("Clean Price = %12.7f" % bond.cleanPriceFromYTM(settlementDate, ytm, yieldConvention))

Accrued interest is accrued from previous coupon date to settlement date

In [ ]:
print("Previous coupon date is ", bond._pcd)

In [ ]:
print("Settlement date is ", settlementDate)

The amount of accrued interest is 

In [ ]:
print("Accrued = %12.2f" % bond._accruedInterest)

This is based on the following number of days of accrual

In [ ]:
print("Accrued Days = %d" % bond._accruedDays)

In [ ]:
print("Principal = %12.2f" % bond.principal(settlementDate, ytm, yieldConvention))

## Bond Risk Measures

The bond dollar duration is the actual derivative with respect to the yield. It is the bond price change for a 1bp drop in the yield-to-maturity divided by 1bp.

In [ ]:
duration = bond.dollarDuration(settlementDate, ytm, yieldConvention)
print("Dollar Duration = ", duration)

Modified Duration divides the dollar duration by the full price of the bond

In [ ]:
modifiedDuration = bond.modifiedDuration(settlementDate, ytm, yieldConvention)
print("Modified Duration = ", modifiedDuration)

Macauley Duration multiplies the dollar duration by (1+y/f) and divides by the full price

In [ ]:
macauleyDuration = bond.macauleyDuration(settlementDate, ytm, yieldConvention)
print("Macauley Duration = ", macauleyDuration)

Convexity is the second derivative of the bond price with respect to the yield-to-maturity

In [ ]:
conv = bond.convexityFromYTM(settlementDate, ytm, yieldConvention)
print("Convexity = ", conv)

Copyright (c) 2020 Dominic O'Kane